### Configuração Inicial

In [1]:
import os
import sys
from pathlib import Path

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add project root to Python path
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

# Now import the modules
from efd_utils.extract_text import extrair_texto, extrair_tabelas
from efd_utils.rag_efd import EFDChunker, EFDVectorStore, EFDChatTransformers, EFDChatOllama, EFDChatBode, limpar_arquivos_temporarios

In [2]:
limpar_arquivos_temporarios()

Limpando cache do HuggingFace...
e:\Carla Reis\Projects\Notebooks\efd-ia\efd_db\vectorstore
Limpando vectorstore...
e:\Carla Reis\Projects\Notebooks\efd-ia\data\txt\validacoes_extraidas.md
Limpando arquivo txt...
e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\chunks.json
Limpando arquivo de chunks...
e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json
Limpando arquivos JSON na pasta json...
✅ Limpeza concluída!


### Extração de dados

In [3]:
# Atualizar o caminho do documento de entrada
input_doc_path = Path(os.getenv('PDF_0200'))

# Extrair texto
extrair_texto(input_doc_path)

# Extrair tabelas
extrair_tabelas(input_doc_path)


✅ Tabela do registro 0190 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_0190.json
✅ Tabela do registro 0200 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_0200.json


In [4]:
input_doc_path = Path(os.getenv('PDF_C170'))

# Extrair texto
extrair_texto(input_doc_path)
    
# Extrair tabelas
extrair_tabelas(input_doc_path)

✅ Tabela do registro C165 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_C165.json
✅ Tabela do registro C170 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_C170.json
✅ Tabela do registro C171 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_C171.json


In [5]:
input_doc_path = Path(os.getenv('PDF_C100'))

# Extrair texto
extrair_texto(input_doc_path)
    
# Extrair tabelas
extrair_tabelas(input_doc_path)

✅ Tabela do registro C001 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_C001.json
✅ Tabela do registro C100 salva em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\json\registro_C100.json


In [6]:
# Cria os chunks
path_data = os.path.join(project_root,"data")
chunker = EFDChunker(path_data +"/txt/validacoes_extraidas.md")
chunks = chunker.extract_registers()
chunker.save_chunks(path_data + "/structured/chunks.json")
        

✅ Campos carregados para o registro 0190: 3 campos
✅ Chunk criado para o registro 0190 com 3 campos
✅ Campos carregados para o registro 0200: 13 campos
✅ Chunk criado para o registro 0200 com 13 campos
✅ Chunk criado para o registro 0205 com 0 campos
✅ Campos carregados para o registro C165: 13 campos
✅ Chunk criado para o registro C165 com 13 campos
✅ Campos carregados para o registro C170: 36 campos
✅ Chunk criado para o registro C170 com 36 campos
✅ Campos carregados para o registro C171: 5 campos
✅ Chunk criado para o registro C171 com 5 campos
✅ Campos carregados para o registro C001: 2 campos
✅ Chunk criado para o registro C001 com 2 campos
✅ Chunks salvos em: e:\Carla Reis\Projects\Notebooks\efd-ia\data\structured\chunks.json


### Indexação no Banco Vetorial

In [7]:
import shutil
import os

embeddings_model = "sentence-transformers/all-MiniLM-L6-v2"

# Cria o vectorstore
vectorstore = EFDVectorStore(path_data + "/structured/chunks.json", embeddings_model)
vectorstore.create_vectorstore()
retriever = vectorstore.vectorstore.as_retriever()

e:\Carla Reis\Projects\Notebooks\efd-ia\efd_utils\rag_efd.py:157: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Vectorstore:  [Document(metadata={'source': 'e:\\Carla Reis\\Projects\\Notebooks\\efd-ia\\data\\txt\\validacoes_extraidas.md', 'register': '0190', 'has_json': True, 'num_campos': 3, 'model_name': 'default-model'}, page_content='IDENTIFICAÇÃO DAS UNIDADES DE MEDIDA\nEste  registro  tem  por  objetivo  descrever  as  unidades  de  medidas  utilizadas  no  arquivo  digital.  Não  podem  ser informados dois ou mais registros com o mesmo código de unidade de medida. Somente devem constar as unidades de medidas informadas em qualquer outro registro.\nObservações:\nNível hierárquico: 2\nOcorrência: vários por arquivo\nCampo 01 (REG) - Valor Válido: [0190]\nCampo 02 (UNID) - Validação: o valor informado neste campo deve existir em, pelo menos, um outro registro do arquivo.\n\nCampos do registro:\n- REG: Texto fixo contendo "0190"\n- UNID: Código da unidade de medida\n- DESCR: Descrição da unidade de medida\n'), Document(metadata={'source': 'e:\\Carla Reis\\Projects\\Notebooks\\efd-ia\\data\\tx

### Avaliação dos LLM x Modelo de Emdeddings `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`

#### Usando LLM Bode

In [8]:
# Cria o chat
chatBode = EFDChatBode()

e:\Carla Reis\Projects\Notebooks\efd-ia\efd_utils\rag_efd.py:388: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm)


In [9]:
# Limite de caracteres por chunk (ajuste conforme necessário)
MAX_CHUNK_CHARS = 300

def limitar_contexto(results, max_chunks=2, max_chars=MAX_CHUNK_CHARS):
    contexto_limitado = []
    for doc in results[:max_chunks]:
        texto = doc.page_content
        if len(texto) > max_chars:
            texto = texto[:max_chars] + "..."
        contexto_limitado.append(
            f"Registro: {doc.metadata.get('register', '')}\nConteúdo: {texto}"
        )
    return "\n\n".join(contexto_limitado)

In [10]:
query = "Quais são os campos do registro 0200?"
results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)

C:\Users\clr_c\AppData\Local\Temp\ipykernel_28376\2873707216.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)
e:\Carla Reis\Projects\Notebooks\efd-ia\efd_utils\rag_efd.py:408: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm_chain.run(instruction)



Pergunta: Quais são os campos do registro 0200?

Resposta: O registro 0200 contém informações de código, nome, descrição e codificação.


In [11]:
query = "Quais são os campos obrigatórios do C170?"
results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os campos obrigatórios do C170?

Resposta: Os campos obrigatórios no C170 incluem o código do registro (C170), o código do item (55) e o volume recebido (em litros).


In [12]:
query = "Quais são os registros relacionados ao campo COD_ITEM?"
results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os registros relacionados ao campo COD_ITEM?

Resposta: Os dois registros relacionados ao campo COD_ITEM são 0205 e 0190.


In [13]:
query = "Quais são os campos do registro C165?"
results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os campos do registro C165?

Resposta: O registro C165 é usado para identificar a unidade de medida e o código de unidade de medida que ocorreram na descrição ou alteração do item.


In [14]:
query = "Qual a regra do campo NUM_ITEM?"

results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Qual a regra do campo NUM_ITEM?

Resposta: O campo NUM_ITEM tem um valor mínimo de 1 e um máximo de 99999.


In [15]:
query = "Qual é a validação do campo 01 do registro C165?"

results = retriever.get_relevant_documents(query)
contexto = limitar_contexto(results, max_chunks=2, max_chars=300)
response = chatBode.get_response(query, contexto)

print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Qual é a validação do campo 01 do registro C165?

Resposta: A validação do campo 01 do registro C165 é verificação de consistência para as linhas que contêm o código de unidade de medida.


#### Usando LLM Llama 3.2

In [16]:
chatLlama = EFDChatOllama()

e:\Carla Reis\Projects\Notebooks\efd-ia\efd_utils\rag_efd.py:344: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [17]:
query = "Quais são os campos do registro 0200?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)

e:\Carla Reis\Projects\Notebooks\efd-ia\efd_utils\rag_efd.py:365: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm([HumanMessage(content=prompt)])



Pergunta: Quais são os campos do registro 0200?

Resposta: Os campos do registro 0200 são:

1. REG - Texto fixo contendo "0200"
2. COD_ITEM - Código do item
3. DESCR_ITEM - Descrição do item
4. COD_BARRA - Representação alfanumérico do código de barra do produto, se houver
5. COD_ANT_ITEM - Código anterior do item com relação à última informação apresentada.
6. UNID_INV - Unidade de medida utilizada na quantificação de estoques.
7. TIPO_ITEM - Tipo do item - Atividades Industriais, Comerciais e Serviços: 00 - Mercadoria para Revenda; 01 - Matéria-prima; 02 - Embalagem; 03 - Produto em Processo; 04 - Produto Acabado; 05 - Subproduto; 06 - Produto Intermediário; 07 - Material de Uso e Consumo; 08 - Ativo Imobilizado; 09 - Serviços; 10 - Outros insumos; 99 - Outras
8. COD_NCM - Código da Nomenclatura Comum do Mercosul
9. EX_IPI - Código EX, conforme a TIPI
10. COD_GEN - Código do gênero do item, conforme a Tabela 4.2.1
11. COD_LST - Código do serviço conforme lista do Anexo I da Lei Comp

In [18]:
query = "Quais são os campos obrigatórios do C170?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os campos obrigatórios do C170?

Resposta: Os campos obrigatórios do registro C170 são:

1. COD_PART
2. VEIC_ID
3. COD_AUT
4. NR_PASSE
5. HORA
6. TEMPER
7. QTD_VOL
8. PESO_BRT
9. PESO_LIQ
10. NOM_MOT
11. CPF
12. UF_ID


In [19]:
query = "Quais são os registros relacionados ao campo COD_ITEM?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os registros relacionados ao campo COD_ITEM?

Resposta: Os registros relacionados ao campo COD_ITEM são os seguintes:

* Registro 0200: REG "0200", Descrição do item, Código do item, Descrição do item e outros campos específicos.
* Registro 0150: REG "0150", Descrição do item, Código do item, Descrição do item e outros campos específicos.
* Registro C165: REG "C165", Código do item, Descrição do item, Código da Nomenclatura Comum do Mercosul e outros campos específicos.

Esses registros contêm informações sobre os itens de negócios e as operações relacionadas a eles.


In [20]:
query = "Quais são os campos do registro C165?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Quais são os campos do registro C165?

Resposta: Os campos do registro C165 são:

1. REG (Texto fixo contendo "C165")
2. COD_PART (Código do participante, que deve existir no campo COD_PART do registro 0150)
3. VEIC_ID (Placa de identificação do veículo)
4. COD_AUT (Código da autorização fornecida pela SEFAZ)
5. NR_PASSE (Número do Passe Fiscal)
6. HORA (Hora da saída das mercadorias, em formato 'hhmmss')
7. TEMPER (Temperatura em graus Celsius utilizada para quantificação do volume de combustível)
8. QTD_VOL (Quantidade de volumes transportados)
9. PESO_BRT (Peso bruto dos volumes transportados em kg)
10. PESO_LIQ (Peso líquido dos volumes transportados em kg)
11. NOM_MOT (Nome do motorista)
12. CPF (CPF do motorista)
13. UF_ID (Sigla da UF da placa do veículo)


In [21]:
query = "Qual a regra do campo NUM_ITEM?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Qual a regra do campo NUM_ITEM?

Resposta: O campo NUM_ITEM deve ser sempre igual à quantidade de registros de C165 que estão sendo relacionados.


In [22]:
query = "Qual é a validação do campo 01 do registro C165?"
results = retriever.get_relevant_documents(query)
        
# Gera a resposta
response = chatLlama.get_response(query, results)
        
print("\nPergunta:", query)
print("\nResposta:", response)


Pergunta: Qual é a validação do campo 01 do registro C165?

Resposta: A validação do campo 01 (REG) do registro C165 é que deve ser "C165".
